In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.1 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubun

In [3]:
import pandas as pd
# import pandasql as ps
# # import pixiedust
# import sys
# sys.path.append('..')
# import util
# import etl
# import pyarrow.parquet as pq
# import pyarrow as pa
# !pip install duckdb
# import duckdb
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
# spark = SparkSession.builder.getOrCreate()

# util.usedatabase(spark, "real_world_data_jun_2022")

# pixiedust.enableJobMonitor()

# con = duckdb.connect()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cohort_new = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort2024')

In [5]:
cohort_demo = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort_new_demo')

In [ ]:
cohort_demo.count()
# 1701685

1701685

In [6]:
cohort_demo.show(5)

+--------------------+--------------------+------+--------------------+-----+
|            personid|           birthdate|gender|                date| race|
+--------------------+--------------------+------+--------------------+-----+
|00000559-6f93-4e9...|{[1970-02-19], 19...|Female|2017-02-02T19:29:...|White|
|00003900-2614-4da...|{[2009-09-05], 20...|  Male|2015-09-27T04:00:...|White|
|0001df3b-fec4-4e9...|{[1984-04-01], 19...|Female| 2016-03-01T00:00:00|White|
|000538ff-d137-48e...|{[1974-10-16], 19...|  Male|2017-06-22T05:00:...| NULL|
|001517b6-1588-409...|{[2015-02-10], 20...|  Male| 2020-07-27T00:00:00|White|
+--------------------+--------------------+------+--------------------+-----+
only showing top 5 rows



In [ ]:
cohort_como = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort_new_como')

In [ ]:
cohort_como.show(5)

+--------------------+--------------------+-------------+--------------------+
|            personid|         encounterid|comorbidityid|       effectivedate|
+--------------------+--------------------+-------------+--------------------+
|00169bfe-c751-491...|fd5f3f12-37ca-418...|       Z34.82|2022-03-21T12:44:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z33.1|2021-11-28T20:07:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        U07.1|2021-11-28T20:13:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z45.2|2022-01-04T15:02:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        E04.9|2022-01-02T17:00:...|
+--------------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [ ]:
cohort_como_reformat= cohort_como.withColumn('comorbidityid',expr("regexp_replace(comorbidityid, '([0-9a-zA-Z]+\\.[0-9a-zA-Z]).*', '$1')").cast("string"))

In [ ]:
cohort_como_reformat.show(5)

+--------------------+--------------------+-------------+--------------------+
|            personid|         encounterid|comorbidityid|       effectivedate|
+--------------------+--------------------+-------------+--------------------+
|00169bfe-c751-491...|fd5f3f12-37ca-418...|        Z34.8|2022-03-21T12:44:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z33.1|2021-11-28T20:07:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        U07.1|2021-11-28T20:13:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z45.2|2022-01-04T15:02:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        E04.9|2022-01-02T17:00:...|
+--------------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [ ]:
def get_top_comorbidity(spark, como, topn):
    top_como = como.groupBy('comorbidityid').count().orderBy(col('count').desc())\
        .select(collect_list('comorbidityid')).collect()[0]\
            .__getitem__('collect_list(comorbidityid)')[0:topn]

#     top_como_coded = ["n"+x.replace('.','_') for x in top_como]
#     out = {top_como[i]:top_como_coded[i] for i in range(len(top_como))} # store in a dictionary
    print(top_como)
    return top_como

In [ ]:
top_comorbidity = get_top_comorbidity(spark, cohort_como_reformat, topn=50)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1']


In [ ]:
cohort_como_length = cohort_como_reformat.join(cohort_new.select('personid','date'),'personid').withColumn('len_como',months_between(col('date'),col('effectivedate'))/12)

In [ ]:
# prompt: make the len_como integer

cohort_como_length = cohort_como_length.withColumn("len_como", col("len_como").cast("integer"))


In [ ]:
cohort_como_length_filter = cohort_como_length.filter(col('comorbidityid').isin(top_comorbidity))

In [ ]:
cohort_como_length_filter.show(5)

+--------------------+--------------------+-------------+--------------------+--------------------+--------+
|            personid|         encounterid|comorbidityid|       effectivedate|                date|len_como|
+--------------------+--------------------+-------------+--------------------+--------------------+--------+
|00010e6e-593a-4e1...|d60bc852-0992-4b1...|        V58.6|2015-08-31T09:06:...|2020-03-17T03:09:...|       4|
|00010e6e-593a-4e1...|c8a0da6a-aadd-47d...|        M79.6|2016-01-29T04:15:...|2020-03-17T03:09:...|       4|
|00010e6e-593a-4e1...|1fbe46d9-25ff-467...|        M54.2|2020-01-14T17:00:...|2020-03-17T03:09:...|       0|
|00010e6e-593a-4e1...|87247b0f-a8f0-45d...|        F17.2|2020-02-28T19:30:...|2020-03-17T03:09:...|       0|
|00010e6e-593a-4e1...|87247b0f-a8f0-45d...|        M54.2|2020-02-28T19:30:...|2020-03-17T03:09:...|       0|
+--------------------+--------------------+-------------+--------------------+--------------------+--------+
only showing top 5 

In [ ]:
# prompt: keep personid, comorbidityid and len_como in cohort_como_length_filter and remove duplicates

cohort_como_length_filter_selected = cohort_como_length_filter.select('personid', 'comorbidityid', 'len_como').dropDuplicates()
# cohort_como_length_filter_selected.show(5)


In [ ]:
cohort_como_length_filter.count()

11159560

In [ ]:
cohort_demo.show(5)

+--------------------+--------------------+------+--------------------+-----+
|            personid|           birthdate|gender|                date| race|
+--------------------+--------------------+------+--------------------+-----+
|00000559-6f93-4e9...|{[1970-02-19], 19...|Female|2017-02-02T19:29:...|White|
|00003900-2614-4da...|{[2009-09-05], 20...|  Male|2015-09-27T04:00:...|White|
|0001df3b-fec4-4e9...|{[1984-04-01], 19...|Female| 2016-03-01T00:00:00|White|
|000538ff-d137-48e...|{[1974-10-16], 19...|  Male|2017-06-22T05:00:...| NULL|
|001517b6-1588-409...|{[2015-02-10], 20...|  Male| 2020-07-27T00:00:00|White|
+--------------------+--------------------+------+--------------------+-----+
only showing top 5 rows



In [7]:
cohort_demo_processed = cohort_demo.withColumn('age_at_diagnosis',months_between(col('date'),col('birthdate.value'))/12)
cohort_demo_processed = cohort_demo_processed.withColumn('age_at_diagnosis',col('age_at_diagnosis').cast("integer"))
cohort_demo_processed = cohort_demo_processed.drop('birthdate')
# cohort_demo_processed.show(5)

In [8]:
# Preprocess gender column
df = cohort_demo_processed
df = df.withColumn("gender",
                   when(lower(col("gender")).isin("male"), 1)
                   .when(lower(col("gender")).isin("female"), 2)
                   .when(col("gender").isNull(), 0)
                   .otherwise(3))

# Preprocess race column
df = df.withColumn("race",
                   when(lower(col("race")).like("%white%") | lower(col("race")).like("%caucasian%"), 1)
                   .when(lower(col("race")).like("%black%") | lower(col("race")).like("%african%"), 2)
                   .when(lower(col("race")).like("%hispanic%"), 3)
                   .when(lower(col("race")).like("%asian%") | lower(col("race")).like("%chinese%") | lower(col("race")).like("%korean%") | lower(col("race")).like("%japanese%"), 4)
                   .when(lower(col("race")).like("%indian american%") | lower(col("race")).like("%native american%"), 5)
                   .when(col("race").isNull(), 0)
                   .otherwise(6))
df.show()
cohort_demo_processed = df

+--------------------+------+--------------------+----+----------------+
|            personid|gender|                date|race|age_at_diagnosis|
+--------------------+------+--------------------+----+----------------+
|00000559-6f93-4e9...|     2|2017-02-02T19:29:...|   1|              46|
|00003900-2614-4da...|     1|2015-09-27T04:00:...|   1|               6|
|0001df3b-fec4-4e9...|     2| 2016-03-01T00:00:00|   1|              31|
|000538ff-d137-48e...|     1|2017-06-22T05:00:...|   0|              42|
|001517b6-1588-409...|     1| 2020-07-27T00:00:00|   1|               5|
|00179a60-4542-4d4...|     1|2020-03-27T11:35:...|   1|              81|
|001a2011-9523-415...|     1|2022-01-08T12:11:...|   1|              26|
|0021fe8a-5e63-441...|     1|2019-11-28T18:00:...|   1|               2|
|0021ff9b-1406-472...|     1|2018-01-11T08:00:...|   1|              26|
|0029b094-5715-477...|     2|                    |   2|            NULL|
|00384f92-4a1a-4e7...|     1|2020-04-14T18:26:...| 

In [9]:
# prompt: join with cohort new

# Assuming 'cohort_new' DataFrame already exists as defined in the previous code.

# Assuming 'cohort_demo_processed' DataFrame already exists as defined in the previous code.

# Join the two dataframes
joined_cohort = cohort_new.join(cohort_demo_processed, on='personid', how='inner')

# Show the first 5 rows of the joined dataframe
joined_cohort.show(5)

+--------------------+--------------------+---+------+--------------------+----+----------------+
|            personid|                date|EPI|gender|                date|race|age_at_diagnosis|
+--------------------+--------------------+---+------+--------------------+----+----------------+
|000084f6-14ea-48c...|2017-06-17T07:00:...|  0|     1|2017-06-17T07:00:...|   6|              38|
|00010e6e-593a-4e1...|2020-03-17T03:09:...|  0|     2|2020-03-17T03:09:...|   1|              31|
|00012328-6d23-405...|2020-05-11T19:00:...|  0|     2|2020-05-11T19:00:...|   1|               2|
|000151d8-40ee-46d...|2021-09-22T18:40:...|  0|     1|2021-09-22T18:40:...|   1|              77|
|0001b472-d009-41b...|2021-06-28T16:00:...|  0|     2|2021-06-28T16:00:...|   2|               1|
+--------------------+--------------------+---+------+--------------------+----+----------------+
only showing top 5 rows



In [15]:
joined_cohort = joined_cohort.select('personid','age_at_diagnosis','gender','race','EPI')

In [20]:
# prompt: for age_at_diagnosis, describe the mean, and 95% confidence interval of age_at_diagnosis; For gender, describe percentile of 0, 1, 2, ; for for race describe percentile of 1, 2, 3, 4, 5, 6.  stratified by different EPI groups (0 or 1)

from pyspark.sql.functions import mean, stddev, count, percentile_approx

# Group by EPI group (assuming 'EPI' column exists)
grouped_cohort = joined_cohort.groupBy("EPI")

# Calculate mean and standard deviation of age_at_diagnosis for each EPI group
summary_stats = grouped_cohort.agg(
    mean("age_at_diagnosis").alias("mean_age"),
    stddev("age_at_diagnosis").alias("stddev_age"),
    count("*").alias("n")
)

# Calculate the 95% confidence interval
summary_stats = summary_stats.withColumn(
    "lower_ci", summary_stats["mean_age"] - 1.96 * (summary_stats["stddev_age"] / (summary_stats["n"] ** 0.5))
).withColumn(
    "upper_ci", summary_stats["mean_age"] + 1.96 * (summary_stats["stddev_age"] / (summary_stats["n"] ** 0.5))
)

summary_stats.show()


+---+-----------------+------------------+-------+-----------------+------------------+
|EPI|         mean_age|        stddev_age|      n|         lower_ci|          upper_ci|
+---+-----------------+------------------+-------+-----------------+------------------+
|  1|43.52829781261472|23.939486199073553| 102687|43.38187351028304| 43.67472211494639|
|  0|32.20221205268794|27.451904461337097|1598998|32.15966156031688|32.244762545059004|
+---+-----------------+------------------+-------+-----------------+------------------+



In [21]:
# prompt: get the distributions of gender for different EPI

from pyspark.sql.functions import count, when

# Assuming 'joined_cohort' DataFrame already exists as defined in the previous code.

# Group by EPI and gender
gender_distribution = joined_cohort.groupBy("EPI", "gender").agg(count("*").alias("count"))

# Show the results
gender_distribution.show()

+---+------+------+
|EPI|gender| count|
+---+------+------+
|  1|     0|    15|
|  1|     2| 44068|
|  1|     1| 58516|
|  1|     3|    88|
|  0|     0|   440|
|  0|     1|854020|
|  0|     2|743119|
|  0|     3|  1419|
+---+------+------+



In [25]:
# prompt: generate race distribution, sort by EPI then race

# Assuming 'joined_cohort' DataFrame already exists as defined in the previous code.

# Group by EPI and race
race_distribution = joined_cohort.groupBy("EPI", "race").agg(count("*").alias("count"))

# Order by EPI then race
race_distribution = race_distribution.orderBy("EPI", "race")

# Show the results
race_distribution.show()

+---+----+-------+
|EPI|race|  count|
+---+----+-------+
|  0|   0|  26543|
|  0|   1|1101705|
|  0|   2| 165850|
|  0|   3|  20257|
|  0|   4|  34026|
|  0|   6| 250617|
|  1|   0|   2006|
|  1|   1|  72049|
|  1|   2|  12403|
|  1|   3|    662|
|  1|   4|   1190|
|  1|   6|  14377|
+---+----+-------+



In [ ]:
person_vertice = cohort_demo_processed
person_vertice = person_vertice.withColumn("id", col('personid')).drop('personid','date')
diagnosis_vertice = cohort_como_length_filter.select("comorbidityid").distinct()
diagnosis_vertice = diagnosis_vertice.withColumn("id", col('comorbidityid')).drop(col('comorbidityid'))

In [ ]:
person_vertice.select('id').count()

1701685

In [ ]:
person_vertice.select('id').distinct().count()

1701685

In [ ]:
# print(person_vertice.columns)
# print(diagnosis_vertice.columns)

In [ ]:
# def union_columns(df1, df2):
#     diff1 = [c for c in df2.columns if c not in df1.columns]
#     diff2 = [c for c in df1.columns if c not in df2.columns]
#     df = df1.select('*', *[lit(None).alias(c) for c in diff1]) \
#         .unionByName(df2.select('*', *[lit(None).alias(c) for c in diff2]))
#     return df

In [ ]:
# All_vertices = union_columns(person_vertice, diagnosis_vertice)

In [ ]:
personid_diagnosis_edges = cohort_new.join(cohort_como_length_filter,'personid').select("personid", "comorbidityid",'len_como')
# personid_diagnosis_edges = personid_diagnosis_edges.withColumn("cormorbidity_year", col('comorbidity_year'))
personid_diagnosis_edges = personid_diagnosis_edges.withColumn("src", col('personid'))
personid_diagnosis_edges = personid_diagnosis_edges.withColumn("dst", col('comorbidityid'))

diagnosis_personid_edges = personid_diagnosis_edges.withColumn("src", col('comorbidityid'))
diagnosis_personid_edges = personid_diagnosis_edges.withColumn("dst", col('personid'))

personid_diagnosis_edges = personid_diagnosis_edges.drop(col('personid')).drop(col('comorbidityid'))
diagnosis_personid_edges = diagnosis_personid_edges.drop(col('personid')).drop(col('comorbidityid'))

In [ ]:
# all_edges = personid_diagnosis_edges.unionAll(diagnosis_personid_edges)
all_edges = personid_diagnosis_edges

In [ ]:
all_edges.show(5)

+--------+--------------------+-----+
|len_como|                 src|  dst|
+--------+--------------------+-----+
|       4|00010e6e-593a-4e1...|V58.6|
|       4|00010e6e-593a-4e1...|M79.6|
|       0|00010e6e-593a-4e1...|M54.2|
|       0|00010e6e-593a-4e1...|F17.2|
|       0|00010e6e-593a-4e1...|M54.2|
+--------+--------------------+-----+
only showing top 5 rows



In [ ]:
# Create input features tensor
# import torch

# # Mapping person and comorbidity IDs to indices
# person_id_to_idx = {id_: idx for idx, id_ in enumerate(person_vertice.toPandas()['id'])}
# comorbidity_id_to_idx = {id_: idx + len(person_id_to_idx) for idx, id_ in enumerate(diagnosis_vertice.toPandas()['id'])}
# id_to_idx = {**person_id_to_idx, **comorbidity_id_to_idx}

In [ ]:
# revised to put comorbidity indexes at the beginning
import torch

# Mapping person and comorbidity IDs to indices
comorbidity_id_to_idx = {id_: idx  for idx, id_ in enumerate(diagnosis_vertice.toPandas()['id'])}
person_id_to_idx = {id_: idx + len(comorbidity_id_to_idx) for idx, id_ in enumerate(person_vertice.toPandas()['id'])}

id_to_idx = {**comorbidity_id_to_idx, **person_id_to_idx}

In [ ]:
# prompt: convert id of person_vertice using id_to_idx

# Assuming 'person_vertice' is a PySpark DataFrame
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Create a UDF to map person IDs to indices
map_person_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'id' column of 'person_vertice'
person_vertice_indexed = person_vertice.withColumn("id_index", map_person_id_udf(col("id")))

# Show the updated DataFrame
# person_vertice_indexed.show()


In [ ]:
# prompt: convert id of diagnosis_vertice using id_to_idx

# Create a UDF to map comorbidity IDs to indices
map_comorbidity_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'id' column of 'diagnosis_vertice'
diagnosis_vertice_indexed = diagnosis_vertice.withColumn("id_index", map_comorbidity_id_udf(col("id")))

# Show the updated DataFrame
# diagnosis_vertice_indexed.show()


In [ ]:
# prompt: convert src and dst of all_edges using id_to_idx

# Create a UDF to map IDs to indices for 'src' column
map_src_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'src' column of 'all_edges'
all_edges_indexed = all_edges.withColumn("src_index", map_src_id_udf(col("src")))

# Create a UDF to map IDs to indices for 'dst' column
map_dst_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'dst' column of 'all_edges_indexed'
all_edges_indexed = all_edges_indexed.withColumn("dst_index", map_dst_id_udf(col("dst")))

# Show the updated DataFrame
# all_edges_indexed.show()


In [ ]:
person_vertice_indexed.show()

+------+----+----------------+--------------------+--------+
|gender|race|age_at_diagnosis|                  id|id_index|
+------+----+----------------+--------------------+--------+
|     2|   1|              46|00000559-6f93-4e9...|      50|
|     1|   1|               6|00003900-2614-4da...|      51|
|     2|   1|              31|0001df3b-fec4-4e9...|      52|
|     1|   0|              42|000538ff-d137-48e...|      53|
|     1|   1|               5|001517b6-1588-409...|      54|
|     1|   1|              81|00179a60-4542-4d4...|      55|
|     1|   1|              26|001a2011-9523-415...|      56|
|     1|   1|               2|0021fe8a-5e63-441...|      57|
|     1|   1|              26|0021ff9b-1406-472...|      58|
|     2|   2|            NULL|0029b094-5715-477...|      59|
|     1|   1|              36|00384f92-4a1a-4e7...|      60|
|     2|   1|              17|003bd84f-f6bd-40a...|      61|
|     2|   1|               1|003eb796-70b0-430...|      62|
|     1|   1|           

In [ ]:
# prompt: convert null in age at diagnosis to 0

person_vertice_indexed = person_vertice_indexed.fillna({'age_at_diagnosis': 0})


In [ ]:
# # prompt: handle null value of x_person

# # Handle null values in x_person
# x_person = torch.nan_to_num(x_person, nan=0)  # Replace NaN with 0.0

# print(x_person)


In [ ]:
# prompt: format person_vertice to x_person, format diagnosis_vertice to x_diagnosis

# Assuming 'person_vertice' is a PySpark DataFrame with columns: 'id_index', 'age_at_diagnosis', 'gender', 'race'
x_person = torch.tensor(person_vertice_indexed.select("id_index", "age_at_diagnosis", "gender", "race")
                         .toPandas().values, dtype=torch.int)

# Assuming 'diagnosis_vertice' is a PySpark DataFrame with column: 'id_index'
x_diagnosis = torch.tensor(diagnosis_vertice_indexed.select("id_index")
                            .toPandas().values, dtype=torch.int)

# print(x_person)
# print(x_diagnosis)


Py4JJavaError: An error occurred while calling o371.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 68.0 failed 1 times, most recent failure: Lost task 6.0 in stage 68.0 (TID 215) (f19afb5276c5 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:106)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:77)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:555)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:172)
	at org.apache.spark.sql.ca

In [ ]:
# prompt: pad x_diagnosis with 0 so that it has the same dimensions except for dimension 0

# Pad x_diagnosis with zeros
padding_size = x_person.shape[1] - x_diagnosis.shape[1]
padding = torch.zeros(x_diagnosis.shape[0], padding_size)
x_diagnosis_padded = torch.cat([x_diagnosis, padding], dim=1).float()

print(x_diagnosis_padded)


In [ ]:
# # Merge person and diagnosis features
# x_combined = torch.cat([x_person, x_diagnosis_padded], dim=0)
# print(x_combined)

In [ ]:
# Merge person and diagnosis features, put diagnosis first
x_combined = torch.cat([x_diagnosis_padded, x_person], dim=0)
print(x_combined)

In [ ]:
# prompt: format all_edges to the structure of edge_index_person_to_diagnosis, the edge_attr will be len_como

# Extract source and destination indices and edge attributes
src_indices = all_edges_indexed.select("src_index").rdd.flatMap(lambda x: x).collect()
# dst_indices = all_edges_indexed.select("dst_index").rdd.flatMap(lambda x: x).collect()
# edge_attrs = all_edges_indexed.select("len_como").toPandas().values



In [ ]:
# dst_indices = all_edges_indexed.select("dst_index").toPandas().values
dst_indices = all_edges_indexed.select("dst_index").rdd.flatMap(lambda x: x).collect()

In [ ]:
# edge_index_person_to_diagnosis = torch.tensor([src_indices, dst_indices]).contiguous()
edge_index_person_to_diagnosis = torch.tensor([src_indices, dst_indices]).contiguous()

print(edge_index_person_to_diagnosis)

In [ ]:
edge_index_diagnosis_to_person = torch.tensor([dst_indices, src_indices]).contiguous()
# edge_index_diagnosis_to_person = torch.tensor([dst_indices[:,0].tolist(), src_indices]).contiguous()
# print(edge_index_person_to_diagnosis)

In [ ]:
edge_attrs = all_edges_indexed.select("len_como").rdd.map(lambda x: x[0]).collect()
# edge_attrs = all_edges_indexed.select("len_como").toPandas().values[:,0].tolist()

# Create edge attribute tensor
edge_attr = torch.tensor(edge_attrs, dtype=torch.float).unsqueeze(1)  # Unsqueeze to match the expected shape
print(edge_attr)

In [ ]:
# prompt: transform the cohort_new so that personid is converted to indexes using id_to_idx. EPI is the label. Make sure the label is aligned with id in x_person. Load the label to y and prepare data to be loaded using dataloader

# Create a UDF to map person IDs to indices for labels
map_person_id_for_label_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'personid' column of 'cohort_new' and select 'EPI' as label
cohort_new_indexed = cohort_new.withColumn("id_index", map_person_id_for_label_udf(col("personid"))).select('id_index','EPI')

# align with x_person by doing sorting
cohort_new_indexed = cohort_new_indexed.orderBy('id_index')
# display(cohort_new_indexed.show(5))

# Extract labels and convert to tensor
y = torch.tensor(cohort_new_indexed.select("EPI").toPandas().values.astype(int), dtype=torch.long).squeeze()

# Ensure labels are aligned with IDs in x_person
# (Assuming x_person is already sorted by person ID index)
# If not, you might need to sort both x_person and y by ID index





In [ ]:
from torch_geometric.data import HeteroData, DataLoader
# Create HeteroData object
data1 = HeteroData()
data1.x = x_combined.to(torch.int64)
data1.edge_index = edge_index_person_to_diagnosis
data1.edge_attr = edge_attr
data1.y = y


In [ ]:
data1 # 6661834

In [ ]:
data1.x

In [ ]:
# prompt: write data1 into h5

import h5py

# Create an h5 file
with h5py.File('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag50_first.h5', 'w') as f:
  # Store node features
  f.create_dataset('x', data=data1.x.numpy())

  # Store edge index
  f.create_dataset('edge_index', data=data1.edge_index.numpy())

  # Store edge attributes
  f.create_dataset('edge_attr', data=data1.edge_attr.numpy())

  # Store labels
  f.create_dataset('y', data=data1.y.numpy())
